In [1]:
%matplotlib inline
import os

In [2]:
import datajoint as dj

In [3]:
import pynwb
from pynwb import NWBFile, NWBHDF5IO
import pathlib
from pipeline import lab, experiment, ephys, psth

Connecting sfn19_user@workshop-db.datajoint.io:3306


In [4]:
from pipeline.export.datajoint_to_nwb import export_to_nwb

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Objective
We wish to create a ***computed*** table storing NWB file, with attribute type of ***filepath***. The idea is to generate NWB files, one for each session, that can be access from the file system, hence type ***filepath***, and can also be fetched and worked with as part of DatJoint pipeline. This can be accomplished with the new ***dj.AttributeAdapter*** feature.

## Create a ***store*** for the filepath

In [16]:
exported_nwb_dir = '/home/ttngu207/data'

In [17]:
dj.config['stores'] = {
    'nwbstore': {'protocol': 'file',
                 'stage': exported_nwb_dir,
                 'location': exported_nwb_dir}}

## Create a DataJoint AttributeAdapter for NWB object

Basically we will need to define an object inhereted from `dj.AttributeAdapter` and instantiated with a variable name ***nwb_obj***

In [10]:
class NWBAdapter(dj.AttributeAdapter):
    attribute_type = 'filepath@nwbstore'
    
    def put(self, nwbfile):
        save_file_name = ''.join([nwbfile.identifier, '.nwb'])
        with NWBHDF5IO(os.path.join(exported_nwb_dir, save_file_name), mode='w') as io:
            io.write(nwbfile)
            print(f'Write NWB 2.0 file: {save_file_name}')
        return os.path.join(exported_nwb_dir, save_file_name)
        
    def get(self, path):
        return NWBHDF5IO(path, mode='r').read()

#### Instantiate for use as a datajoint type

In [11]:
nwb_obj = NWBAdapter()

## Create a new schema ***export*** and NWB table

In [7]:
export = dj.schema('djneuro_sfn19_export')

In [11]:
@export
class NWB(dj.Manual):
    definition = """
    -> experiment.Session
    ---
    nwb: <nwb_obj> 
    """

Note that the table definition above set the ***nwb*** attribute to be of type ***< nwb_obj >***. Hence the reason for defining ***nwbfile*** as an instant of ***NWBAdapter***

## Build an NWBFile and insert into ***NWB** table

In [12]:
experiment.Session()

subject_id institution 6 digit animal ID,session session number,session_date,username,rig
210861,1,2013-07-01,Nuo Li,None
210861,2,2013-07-02,Nuo Li,None
210861,3,2013-07-03,Nuo Li,None
210862,1,2013-06-26,Nuo Li,None
210862,2,2013-06-27,Nuo Li,None
210862,3,2013-06-28,Nuo Li,None
210863,1,2013-06-26,Nuo Li,None


In [13]:
session_key = (experiment.Session & {'subject_id': '210861', 'session': 2}).fetch1('KEY')

In [14]:
nwbfile = export_to_nwb(session_key, nwb_output_dir=exported_nwb_dir, save=False, overwrite=True)

Exporting to NWB 2.0 for session: {'subject_id': 210861, 'session': 2, 'session_date': datetime.date(2013, 7, 2), 'username': 'Nuo Li', 'rig': None}...


/opt/conda/lib/python3.7/site-packages/pynwb/file.py:621: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
/opt/conda/lib/python3.7/site-packages/pynwb/file.py:621: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


In [15]:
nwbfile


root <class 'pynwb.file.NWBFile'>
Fields:
  acquisition: {
    BehavioralEvents <class 'pynwb.behavior.BehavioralEvents'>,
    BehavioralTimeSeries <class 'pynwb.behavior.BehavioralTimeSeries'>
  }
  devices: {
    A4x8-5mm-100-200-177 <class 'pynwb.device.Device'>,
    LaserGem473 <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    silicon32_g0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'pynwb.core.DynamicTable'>
  experimenter: Nuo Li
  institution: Janelia Research Campus
  ogen_sites: {
    left_alm <class 'pynwb.ogen.OptogeneticStimulusSite'>,
    left_pons <class 'pynwb.ogen.OptogeneticStimulusSite'>
  }
  subject: subject <class 'pynwb.file.Subject'>
  trials: trials <class 'pynwb.epoch.TimeIntervals'>
  units: units <class 'pynwb.misc.Units'>
  virus: [{"injection_id": "1", "virus": "Addgene41015", "injection_date": "2013-05-23", "injection_volume": "30.0", "brain_location_name": "left_m2", "ml_location": "2500.0", "ap_location": "-1500.

### Check the units and spikes

In [16]:
nwbfile.units.to_dataframe()

,quality,posx,posy,amp,snr,cell_type,spike_times,electrodes,electrode_group,waveform_mean,waveform_sd
id,,,,,,,,,,,
2,good,-100.0,-550.267,NaN,NaN,Pyr,"[109.26314320573425, 121.17209161397552, 125.2...","[(20, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.9228864537202256e-05, 2.84284926345073e-05,...","[1.7966140626141845e-05, 1.845440552697878e-05..."
3,good,100.0,-350.267,NaN,NaN,Pyr,"[120.8412642585907, 123.83551503297424, 144.59...","[(10, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[5.006836949003987e-05, 4.427708409176494e-05,...","[2.141107841072877e-05, 1.9222327053230733e-05..."
4,good,-100.0,-450.267,NaN,NaN,Pyr,"[1126.92128716378, 1127.0913846936683, 1130.81...","[(19, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[1.7900644338382988e-05, 1.881759288214055e-05...","[1.4179425243004403e-05, 1.3871909939513117e-0..."
5,good,-100.0,-350.267,NaN,NaN,Pyr,"[109.45957138547516, 111.13029088983154, 111.2...","[(18, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[1.4178244659691839e-05, 1.4123740665604622e-0...","[1.3276078125989425e-05, 1.2994364926017598e-0..."
6,good,-100.0,-650.267,NaN,NaN,Pyr,"[108.79099216470337, 108.88044347772217, 108.9...","[(21, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.2559186228204775e-05, 2.2324021552223898e-0...","[1.8439354724337774e-05, 1.846228827506651e-05..."
7,good,-300.0,-550.267,NaN,NaN,Pyr,"[108.32261978247976, 108.32496384689426, 108.3...","[(28, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.5274703117986277e-05, 2.412432922773945e-05...","[2.306540646069597e-05, 2.3055564978177688e-05..."
10,good,-300.0,-850.267,NaN,NaN,Pyr,"[674.9961780285645, 721.4571693060608, 721.490...","[(31, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.1812180387342468e-05, 1.948675954720962e-05...","[1.9222300473052763e-05, 1.9481898313880234e-0..."
11,good,-300.0,-350.267,NaN,NaN,Pyr,"[132.8796546395569, 284.34961899320984, 284.50...","[(26, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[1.4399569039739552e-05, 1.331808914530563e-05...","[1.2683150482767073e-05, 1.253133861857105e-05..."
12,good,300.0,-450.267,NaN,NaN,Pyr,"[108.86798658380127, 108.89053698787308, 109.0...","[(3, nan, nan, nan, -1.0, {""ml_location"": ""250...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[6.918093181143515e-06, 6.9638174803832934e-06...","[1.4247289647013013e-05, 1.3919138637092663e-0..."


### Insert to the ***NWB*** table

In [31]:
NWB.insert1({**session_key, 'nwb': nwbfile})

Write NWB 2.0 file: ANM210861_2013-07-02_2.nwb


In [32]:
NWB()

subject_id institution 6 digit animal ID,session session number,nwb
210861,2,=BLOB=


In [33]:
fetched_nwb = (NWB & session_key).fetch1('nwb')

In [34]:
fetched_nwb


root <class 'pynwb.file.NWBFile'>
Fields:
  acquisition: {
    BehavioralEvents <class 'pynwb.behavior.BehavioralEvents'>,
    BehavioralTimeSeries <class 'pynwb.behavior.BehavioralTimeSeries'>
  }
  devices: {
    A4x8-5mm-100-200-177 <class 'pynwb.device.Device'>,
    LaserGem473 <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    silicon32_g0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'pynwb.core.DynamicTable'>
  experimenter: Nuo Li
  institution: Janelia Research Campus
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  ogen_sites: {
    left_alm <class 'pynwb.ogen.OptogeneticStimulusSite'>,
    left_pons <class 'pynwb.ogen.OptogeneticStimulusSite'>
  }
  subject: subject <class 'pynwb.file.Subject'>
  trials: trials <class 'pynwb.epoch.TimeIntervals'>
  units: units <class 'pynwb.misc.Units'>
  virus: [{"injection_id": "1", "virus": "Addgene41015", "injection_date": "2013-05-23", "injection_volume": "30.0", "brain_location

### Confirmed the fetched NWB file ***units*** table

In [35]:
fetched_nwb.units.to_dataframe()

,quality,posx,posy,amp,snr,cell_type,spike_times,electrodes,electrode_group,waveform_mean,waveform_sd
id,,,,,,,,,,,
2,good,-100.0,-550.267,NaN,NaN,Pyr,"[109.26314320573425, 121.17209161397552, 125.2...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.9228864537202256e-05, 2.84284926345073e-05,...","[1.7966140626141845e-05, 1.845440552697878e-05..."
3,good,100.0,-350.267,NaN,NaN,Pyr,"[120.8412642585907, 123.83551503297424, 144.59...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[5.006836949003987e-05, 4.427708409176494e-05,...","[2.141107841072877e-05, 1.9222327053230733e-05..."
4,good,-100.0,-450.267,NaN,NaN,Pyr,"[1126.92128716378, 1127.0913846936683, 1130.81...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[1.7900644338382988e-05, 1.881759288214055e-05...","[1.4179425243004403e-05, 1.3871909939513117e-0..."
5,good,-100.0,-350.267,NaN,NaN,Pyr,"[109.45957138547516, 111.13029088983154, 111.2...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[1.4178244659691839e-05, 1.4123740665604622e-0...","[1.3276078125989425e-05, 1.2994364926017598e-0..."
6,good,-100.0,-650.267,NaN,NaN,Pyr,"[108.79099216470337, 108.88044347772217, 108.9...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.2559186228204775e-05, 2.2324021552223898e-0...","[1.8439354724337774e-05, 1.846228827506651e-05..."
7,good,-300.0,-550.267,NaN,NaN,Pyr,"[108.32261978247976, 108.32496384689426, 108.3...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.5274703117986277e-05, 2.412432922773945e-05...","[2.306540646069597e-05, 2.3055564978177688e-05..."
10,good,-300.0,-850.267,NaN,NaN,Pyr,"[674.9961780285645, 721.4571693060608, 721.490...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.1812180387342468e-05, 1.948675954720962e-05...","[1.9222300473052763e-05, 1.9481898313880234e-0..."
11,good,-300.0,-350.267,NaN,NaN,Pyr,"[132.8796546395569, 284.34961899320984, 284.50...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[1.4399569039739552e-05, 1.331808914530563e-05...","[1.2683150482767073e-05, 1.253133861857105e-05..."
12,good,300.0,-450.267,NaN,NaN,Pyr,"[108.86798658380127, 108.89053698787308, 109.0...",None,\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[6.918093181143515e-06, 6.9638174803832934e-06...","[1.4247289647013013e-05, 1.3919138637092663e-0..."


## One step further, let's turn this routine into a dj.Computed table

In [18]:
@export
class ComputedNWB(dj.Computed):
    definition = """
    -> experiment.Session
    ---
    nwb: <nwb_obj> 
    """
    
    def make(self, key):
        exported_nwb_dir = '/home/ttngu207/data'
        nwbfile = export_to_nwb(key, nwb_output_dir=exported_nwb_dir, save=False, overwrite=True)
        self.insert1({**key, 'nwb': nwbfile})

In [ ]:
ComputedNWB.populate()

Exporting to NWB 2.0 for session: {'subject_id': 210861, 'session': 1, 'session_date': datetime.date(2013, 7, 1), 'username': 'Nuo Li', 'rig': None}...
